# Predicting Newsletter Subscription Based on Player Behaviour
## Introduction
**Background**: UBC's Pacific Laboratory for Artificial Intelligence (PLAI) research group runs a Minecraft server called PLAICraft to study player behaviour. They want to know what player traits and behaviours are linked to subscribing to a newsletter.

**Research Question**: Can we predict whether a player will subscribe to a newsletter based on their demographics and gameplay behaviour?

In [4]:
# Load libraries
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## Data Loading

In [46]:
# Load data
players <- read_csv("data/players.csv")
sessions <- read_csv("data/sessions.csv")

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


## Data Wrangling

In [50]:
# Compute session duration
sessions <- sessions |>
    mutate(duration_minutes = as.numeric(as_datetime(original_end_time / 1000) - as_datetime(original_start_time / 1000)) / 60)

# Summary table for player data
session_summary <- sessions |>
    group_by(hashedEmail) |>
    summarize(total_sessions = n(),
             total_minutes_played = sum(duration_minutes, na.rm = TRUE),
             avg_session_duration = mean(duration_minutes, na.rm = TRUE))

# Merge with player data
players_merge <- players |>
    left_join(session_summary, by = "hashedEmail") |>
    mutate(
        total_sessions = replace_na(total_sessions, 0),
        total_minutes_played = replace_na(total_minutes_played, 0),
        avg_session_duration = replace_na(avg_session_duration, 0),
        total_hours_played = total_minutes_played / 60
    )

experience subscribe
1   Pro         TRUE    
2   Veteran     TRUE    
3   Veteran    FALSE    
4   Amateur     TRUE    
5   Regular     TRUE    
6   Amateur     TRUE    
7   Regular     TRUE    
8   Amateur    FALSE    
9   Amateur     TRUE    
10  Veteran     TRUE    
11  Veteran     TRUE    
12  Pro         TRUE    
13  Beginner    TRUE    
14  Amateur     TRUE    
15  Veteran     TRUE    
16  Amateur    FALSE    
17  Beginner    TRUE    
18  Amateur     TRUE    
19  Amateur     TRUE    
20  Regular     TRUE    
21  Regular     TRUE    
22  Amateur    FALSE    
23  Beginner    TRUE    
24  Beginner    TRUE    
25  Amateur     TRUE    
26  Regular     TRUE    
27  Regular     TRUE    
28  Veteran    FALSE    
29  Amateur     TRUE    
30  Veteran    FALSE    
⋮   ⋮          ⋮        
167 Amateur    FALSE    
168 Beginner   FALSE    
169 Veteran     TRUE    
170 Veteran    FALSE    
171 Pro        FALSE    
172 Beginner   FALSE    
173 Veteran     TRUE    
174 Regular    FALSE    
175 Veteran     TRUE    
176 Amateur     TRUE    
177 Pro        FALSE    
178 Veteran    FALSE    
179 Amateur     TRUE    
180 Veteran     TRUE    
181 Amateur    FALSE    
182 Amateur     TRUE    
183 Pro        FALSE    
184 Amateur     TRUE    
185 Pro        FALSE    
186 Regular    FALSE    
187 Veteran     TRUE    
188 Amateur     TRUE    
189 Beginner    TRUE    
190 Amateur    FALSE    
191 Amateur     TRUE    
192 Amateur     TRUE    
193 Veteran    FALSE    
194 Amateur    FALSE    
195 Amateur    FALSE    
196 Pro         TRUE    
    hashedEmail                                                     
1   f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d
2   f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9
3   b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28
4   23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5
5   7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e
6   f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977
7   8e594b8953193b26f498db95a508b03c6fe1c24bb5251d392c18a0da9a722807
8   1d2371d8a35c8831034b25bda8764539ab7db0f63938696917c447128a2540dd
9   8b71f4d66a38389b7528bb38ba6eb71157733df7d1740371852a797ae97d82d1
10  bbe2d83de678f519c4b3daa7265e683b4fe2d814077f9094afd11d8f217039ec
11  7a4686586d290c67179275c7c3dfb4ea02f4d317d9ee0e2cee98baa27877a875
12  4caa42e1b20511552434978171dcf7283fb6eb857eb871327e39bfc328002309
13  4bed6f49b672974f706aa7f020b4f29020326bf594520593557cd381a51559b8
14  3caa832978e0596779f4ee7c686c4592fb6de893925025d25bf26be9a8cd03f5
15  6f9acf8ea9956fe817895c78d10e1e25c11aba335a451e5b0322b073ee5008c4
16  2313a06afe47eacc28ff55adf6f072e7d12b0d12d7cbae29656f9029ec72bb53
17  9db37a1d381697f6c83e3b97b276e38bb9d9c811ba4596edb75d4dbfea6415b0
18  a175d4741dc84e6baf77901f6e8e0a06f54809a34e6b5211159bced346f7fb3e
19  ab1f44f93c3b828f55458971db393052d9711df3e0e7ff69540bfebfcec555ff
20  20ae3f8b5323674fe6e537bc91928841cfc2a00c1b36522fd25370bb6fa89d36
21  bc704ff2bc676dbf48ee41b9e11481c1387bf758ad318f2428f336e3fecc6660
22  200db0417ee70863c7e2a0f214c737476fb86de5869d0b907a077d726357e687
23  f9ac013b2f0bc2bd4928a6a0fc8a0aae8b5c4f2670cf9e18f547b7e72034c965
24  89c8e07d6a2aa102d9cd05086d51b2e41195c9c807dd976a06f4262ff36d0cca
25  119f01b9877fc5ea0073d05602a353b91c4b48e4cf02f42bb8d661b46a34b760
26  5baba1651a0b92788bc0d6dcdf00be64af1cf9f0015bbeed204ddabd1d872e83
27  50cec73a632e4be7b155c3df5165de28b2c5baf092abcb7f509f5645776a56c2
28  f8acd26a5e452b682b9f8b20108caef1ee2e745efe08e95806980bd2388a3fea
29  4b01bce3f141289709e8278b02ba5d2aaa7105d7ccb9c7deb37670a80e332774
30  951e54f7376e2b2f0915e9e3646c701af4a2fe839385b13f9e0084b08f8070bd
⋮   ⋮                                                               
167 7096b30972a32029244fc6171f21bc5996b51b8d75df7a94f003dead428988f9
168 42eafe96ed5c1684e3b5cc614d1b01a117173d3ec6898abcf1177d9822bd39f7
169 9ba67e127e4e70d909f6237644bb4674b17bc6ae5c9e735f88cfc50e22b499c6
170 4bfad3613c71ace05644bf210195d9fb0d3

In [ ]:
#Join with Player Data
player_join <- 